In [68]:
import pandas as pd
import numpy as np
from collections import Counter
import svm
import json

In [69]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function splits a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """
    
    return message.lower().split()
    

In [70]:
def create_dictionary(messages):
    
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message. 

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    msg_list=[]
    for message in messages:
        msg_list.extend(get_words(message))
    d=dict(Counter(msg_list))   
    fil=list(dict(filter(lambda x:x[1]>=5,d.items())).keys())  
    dictionary={w:i for i,w in enumerate(fil)}

            
            
    return dictionary


In [71]:
def transform_text(messages,word_dictionary):
    
     """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each 
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that 
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
        
        
    m,wd=len(messages),len(word_dictionary)
    words_arr=np.zeros((m,wd))
    for ind_m,message in enumerate(messages):
        words=get_words(message)
        for word in words:
            if word in word_dictionary :
                words_arr[ind_m,word_dictionary[word]]+=1
                
    return words_arr
                
            


In [72]:
def fit_naive_bayes_model(matrix,labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    
    labels.reshape((len(labels),1))
    m,V=np.shape(matrix)
    phi_k=np.zeros((2,V))
    phi_y=np.mean(labels)
    phi_k[0]=((np.sum(matrix[labels==0],axis=0))+1)/((np.sum((np.sum(matrix[labels==0],axis=1)),axis=0))+V)
    phi_k[1]=((np.sum(matrix[labels==1],axis=0))+1)/((np.sum((np.sum(matrix[labels==1],axis=1)),axis=0))+V)
    
    return phi_k,phi_y
    


In [73]:
def predict_from_naive_bayes_model(model, matrix):
    
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: The trained model
    """
    
    
    m,V=np.shape(matrix)
    phi_k,phi_y=model
    phi_k_y0=np.sum((np.log(phi_k[0])*matrix),axis=1)
    phi_k_y1=np.sum((np.log(phi_k[1])*matrix),axis=1)
    y_pred=1/(1+np.exp(phi_k_y0+np.log((1-phi_y))-phi_k_y1-np.log(phi_y)))
    y_pred=np.where(y_pred>=0.5,1,0)
    return y_pred





In [84]:
def get_top_five_naive_bayes_words(model, dictionary):
   
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    
    phi_k,phi_y=model
    sort=np.argsort(np.log(phi_k[1]/phi_k[0]))[-1:-6:-1]
    di={i:w for w,i in (dictionary).items()}
    top_five_words=[di[i] for i in sort]
    return top_five_words


In [85]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        eval_matrix: The word counts for the validation data
        eval_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    
    train_labels.reshape((len(train_labels),1))
    val_labels.reshape((len(val_labels),1))
    
    acc1=0
    for radius in radius_to_consider:
            y_pred=svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius)
            acc=(np.mean(y_pred==val_labels))*100
            if acc>acc1:
                acc1=acc
                best_svm_radius=radius
    return best_svm_radius

        

In [86]:
def main():
    data_tr=pd.read_csv(r'ds6_train.tsv',sep='\t',header=None,names=['y','x'])
    data_val=pd.read_csv(r'ds6_val.tsv',sep='\t',header=None,names=['y','x'])
    data_test=pd.read_csv(r'ds6_test.tsv',sep='\t',header=None,names=['y','x'])
    
    messages_tr=data_tr.iloc[:,1].values
    labels_tr=data_tr.iloc[:,0].values
    labels_tr=np.where(labels_tr=='spam',1,0)
    
    messages_val=data_val.iloc[:,1].values
    labels_val=data_val.iloc[:,0].values
    labels_val=np.where(labels_val=='spam',1,0)
    
    messages_test=data_test.iloc[:,1].values
    labels_test=data_test.iloc[:,0].values
    labels_test=np.where(labels_test=='spam',1,0)
    
    dictionary=create_dictionary(messages_tr)
    matrix_tr=transform_text(messages_tr,dictionary)
    
    with open('prob_6a_dicitonary','w') as f:
        json.dump(dictionary,f)
        
    np.savetxt('prob_6a_sample_train_matrix',matrix_tr[:150,:])

    model=fit_naive_bayes_model(matrix_tr,labels_tr)
    
    matrix_val=transform_text(messages_val,dictionary)
    matrix_test=transform_text(messages_test,dictionary)
    

    y_pred_test=predict_from_naive_bayes_model(model,matrix_test)
    
    
    np.savetxt('prob_6b_naive_bayes_predictions',y_pred_test)
    
    acc=(np.mean(y_pred_test==labels_test))*100
    
    print(f'The Naive Bayes spam classifier had an accuracy of {acc}% on the test set')
    
    top_five_naive_bayes_words=get_top_five_naive_bayes_words(model, dictionary)
    print(f'The top five words indicative of the spam class are {top_five_naive_bayes_words}')
    
    with open('prob_6c_top_indicative_words','w') as tp:
        json.dump(top_five_naive_bayes_words,tp)
   
    
    optimal_radius=compute_best_svm_radius(matrix_tr, labels_tr, matrix_val, labels_val, [0.01, 0.1, 1, 10])
    print(f'The optimal RBF Kernel radius is {optimal_radius}')
    
    
    with open('prob_6d_optimal_radius','w') as op:
        json.dump(optimal_radius,op)
    
    svm_pred=svm.train_and_predict_svm(matrix_tr,labels_tr, matrix_test,optimal_radius)
 
    svm_acc=(np.mean(svm_pred==labels_test))*100
    
    print(f'Accuracy of SVM spam classifier with optimal radius {optimal_radius} is {svm_acc}%')

In [87]:
if __name__=='__main__':
    main()


The Naive Bayes spam classifier had an accuracy of 97.84946236559139% on the test set
The top five words indicative of the spam class are ['claim', 'won', 'prize', 'tone', 'urgent!']
The optimal RBF Kernel radius is 0.1
Accuracy of SVM spam classifier with optimal radius 0.1 is 96.95340501792114%
